<a href="https://colab.research.google.com/github/Madhu-712/Agentic-Patterns-/blob/main/4_Custom_Tool_Use_Agentic_Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [ ]:
from crewai import Agent, Task, Crew

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [ ]:
import os
# from utils import get_openai_api_key, pretty_print_result
# from utils import get_serper_api_key

# openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='/home/badrinarayan/Downloads/instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [ ]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [ ]:
from textblob import TextBlob

class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Perform sentiment analysis using TextBlob
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity

        # Determine sentiment based on polarity
        if polarity > 0:
            return "positive"
        elif polarity < 0:
            return "negative"
        else:
            return "neutral"

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
        "Use sentiment analysis tool to get the tone, if positive go ahead."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
To conduct an in-depth analysis of DeepLearningAI, I need to gather information about the company, focusing on key personnel, recent milestones, and potential needs. I'll start by searching the internet for relevant information about DeepLearningAI.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Deeplearning.ai - Crunchbase Company P

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

**Email Draft 1:**

Subject: Collaborating to Enhance "Generative AI for Everyone"

Dear Andrew,

I hope this message finds you well. I recently came across the exciting news about DeepLearningAI's launch of the "Generative AI for Everyone" course on Coursera. This initiative beautifully aligns with your mission to make AI education accessible to all, and I commend you for this remarkable step forward.

At CrewAI, we share your passion for democratizing AI knowledge and are eager to explore how our solutions can further empower DeepLearningAI's goals. Our AI-driven educational tools are designed to enhance learning experiences, making them more engaging and scalable, which could be a perfect complement to your new course offerings.

We understand the importance of community engagement in education, and our tools facilitate active interaction, fostering a vibrant learning community. We believe that by integrating our solutions, DeepLearningAI can enhance its platform's scalability and reach, allowing even more learners to benefit from your outstanding courses.

I would love the opportunity to discuss how we can collaborate to bring even more value to your learners. Are you available for a quick call next week to explore this further?

Looking forward to the possibility of working together.

Warm regards,

[Your Name]  
Lead Sales Representative  
CrewAI

**Email Draft 2:**

Subject: Elevating AI Education Together

Dear Andrew,

Congratulations on the successful launch of the "Generative AI for Everyone" course! This is a testament to DeepLearningAI's commitment to breaking barriers in AI education and empowering a diverse range of learners.

At CrewAI, we are inspired by your vision and believe we can support this mission by providing innovative AI solutions that enhance educational delivery and engagement. Our technology focuses on creating adaptive learning environments that can seamlessly integrate with your current platforms, providing a scalable solution to meet growing demands.

Additionally, our tools are designed to enhance community engagement, something we know is crucial to DeepLearningAI’s objectives. By fostering interactive and collaborative learning experiences, we can help build a thriving community around your educational initiatives.

I’m keen to discuss how our solutions can add value to your new and existing projects. Could we schedule a meeting to explore potential collaboration opportunities?

Thank you for considering this partnership. I look forward to your positive response.

Best regards,

[Your Name]  
Lead Sales Representative  
CrewAI

**Email Draft 3:**

Subject: Partnering for Greater Impact in AI Education

Hi Andrew,

I hope you’re doing well! I’m thrilled to hear about the "Generative AI for Everyone" course you’ve launched on Coursera. This effort is truly commendable and aligns perfectly with the global need for more inclusive AI education.

CrewAI is at the forefront of developing tools that expand the reach and effectiveness of educational content. Our solutions are uniquely designed to enhance learning experiences by making them more interactive and scalable, which can be particularly beneficial as you continue to expand your course offerings.

Furthermore, we understand the critical role of community in learning. Our platform can enhance community interaction and engagement, helping to build a supportive network of learners and mentors within DeepLearningAI.

Would you be open to a brief discussion next week to explore how we can collaborate to enhance your educational impact even further? I am confident that together we can achieve remarkable outcomes.

Thank you for your time, and I hope to hear from you soon.

Sincerely,

[Your Name]  
Lead Sales Representative  
CrewAI